In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    groq_api_key = 'gsk_**************',
    # other params...
)

In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.michaelpage.co.in/job-detail/embedded-ai/ref/jn-062025-6770844")
page_data = loader.load().pop().page_content
print(page_data)

Embedded AI - JN-062025-6770844 | Michael Page   Skip to main navigation     Mobile Menu      English   Job Search   Save jobs, 0 currently saved jobs  0  Save jobs   Register / Sign in      Job Search   Save jobs, 0 currently saved jobs  0  Save jobs   Register / Sign in  NG Main Navigation Job Seekers   Job Seekers Start your job searchBrowse our jobs and apply for your next role.Find jobs  Find a Job   Job Opportunities  Technology Jobs Sales Jobs Work from Home or Hybrid Jobs Returning to India Pre-MBA Roles View all jobs  Resources  Salary Guide Salary Comparison Tool Job Match Resume Templates & More Create an Account Submit your CV  Contract or Temp Jobs   View Contracting Jobs  Tech Contracting Jobs View all contracting jobs Contracting Resources  5 clear signs a contracting career is for you Contracting vs permanent job: Which is better for you?  Career Advice   Career Tips  Changing Jobs Career Progression Salary Negotiation Tech Job Descriptions Explore all categories Most P

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

<think>
Okay, so I need to extract job postings from the given scraped text and format them into JSON with specific keys: role, experience, skills, and description. Let me start by reading through the text carefully to identify all the job postings.

First, I notice that the text has a section labeled "Embedded AI - JN-062025-6770844 | Michael Page". This seems like a job title along with a reference number. The job is in Bangalore, and it's a temporary position with a salary range of INR100,000 - INR101,000 per year. 

Looking further, under "Your Responsibilities," there are several bullet points. These outline the tasks the role entails, such as developing embedded machine learning models, ensuring performance and efficiency, integrating ML algorithms, and collaborating with engineers. I'll need to include these as the job description.

Next, under "Our Requirements," there are details about the qualifications needed. This includes a degree in Computer Science or related fields, pro

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Embedded AI',
 'experience': '2+ years',
 'skills': ['Degree in Computer Science, Electrical Engineering, or related field',
  'Strong proficiency in Python, C/C++, and embedded programming',
  'Experience with embedded ML frameworks (e.g., TensorFlow Lite, ONNX, Edge Impulse)',
  'Understanding of real-time embedded systems and microcontroller architectures'],
 'description': ['Develop, optimize, and deploy embedded machine learning models',
  'Ensure performance, memory optimization, and energy efficiency of AI solutions',
  'Integrate ML algorithms with embedded controllers and edge computing platforms',
  'Collaborate closely with automation and software engineers to translate AI solutions into production-grade embedded systems']}

In [14]:
type(json_res)

dict

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [16]:
job = json_res
job['skills']

['Degree in Computer Science, Electrical Engineering, or related field',
 'Strong proficiency in Python, C/C++, and embedded programming',
 'Experience with embedded ML frameworks (e.g., TensorFlow Lite, ONNX, Edge Impulse)',
 'Understanding of real-time embedded systems and microcontroller architectures']

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

<think>
Okay, I need to help the user by writing a cold email as Mohan from AtliQ. The job description is for an Embedded AI role, so the email should highlight AtliQ's capabilities in that area.

First, I'll start with a strong subject line that grabs attention, maybe something about expertise in embedded AI solutions.

In the opening, I'll address the hiring manager and express interest in the role. I should mention AtliQ's experience in AI and software consulting, emphasizing their track record with enterprises.

Next, I'll outline why AtliQ is a good fit. I'll break it down into bullet points for clarity. Points should include expertise in Python, C/C++, and embedded programming, experience with frameworks like TensorFlow Lite, and their work with microcontrollers.

Then, I'll showcase their portfolio. From the provided links, the most relevant ones are the ML-Python portfolio, so I'll include that link. I'll mention specific projects they've done, like optimizing ML models for edg